In [1]:
import pandas
import datetime
import feather
import numpy
import scipy.optimize
from aacgmv2 import convert_mlt
import seaborn as sns
from davitpy import utils
from imagers.ssusi import ssusi_utils
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib.dates import date2num, DateFormatter
%matplotlib inline

In [2]:
# Some constants
eventDate = datetime.datetime(2016,7,25)
timeStart = datetime.datetime(2016,7,25,3,0)
timeEnd = datetime.datetime(2016,7,25,7,0)
sapsTime = datetime.datetime(2016,7,25,5,30)
sapsRadList = [ "cvw", "cve","fhw",\
               "fhe", "bks", "wal" ]
selBeam = 7
sapsMlatRange = [ 52., 63. ]#[ 56., 60. ]
sapsVelCutoff = 50.
sapsNpointsCutoff = 0 # per radar per beam
velScale = [ -500, 500 ]
velMagnScale = [ 0, 1500 ]
coords="mag"
# TEC variables
tecScale = [ 0, 10 ]

In [3]:
def round2(number):
    """
    Round a number to the closest half integer.
    """
    return round(number * 2) / 2

def get_west_vel(row):
    westVel = row['vLos']/( numpy.cos(\
                 numpy.deg2rad( 90.-\
                    row['azimCalcMag'] ) ) )
    if westVel >= 2000.:
        return numpy.nan
    return westVel


def convert_to_datetime(row):
    currDateStr = str( int( row["dateStr"] ) )
#     return currDateStr
    if row["timeStr"] < 10:
        currTimeStr = "000" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 100:
        currTimeStr = "00" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 1000:
        currTimeStr = "0" + str( int( row["timeStr"] ) )
    else:
        currTimeStr = str( int( row["timeStr"] ) )
    return datetime.datetime.strptime( currDateStr\
                    + ":" + currTimeStr, "%Y%m%d:%H%M" )


def get_mlon_from_mlt(row):
    if row["normMlt"] < 0: 
        currMLT = row["normMlt"] + 24.
    else:
        currMLT = row["normMlt"]
    if row["endPtnormMlt"] < 0: 
        currEndptMLT = row["endPtnormMlt"] + 24.
    else:
        currEndptMLT = row["endPtnormMlt"]
    row["Mlon"] = numpy.round( \
                        convert_mlt( currMLT, row["date"] , m2a=True ) )
    if row["Mlon"] > 180.:
        row["Mlon"] -= 360.
    row["EndptMlon"] = numpy.round( \
                        convert_mlt( currEndptMLT, row["date"] , m2a=True ) )
    if row["EndptMlon"] > 180.:
        row["EndptMlon"] -= 360.
    return row

In [8]:
# read from the feather file
velsDF = feather.read_dataframe('../data/saps-vps-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
velsDF["velMagn"] = numpy.abs(velsDF["vLos"])
velsDF.head()

,dateStr,timeStr,beam,range,azimCalcMag,vLos,power,spwdth,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,velMagn
0,20160725,200,0,0,-31.8579,0.0000,11.3374,37.2005,38.6897,-146.2574,11.4109,43.7946,142.2906,41,hkw,2016-07-25 02:00:00,11.4109,0.0000
1,20160725,200,0,1,-35.2038,0.0000,11.3374,46.0231,39.0370,-146.7567,11.3776,44.1381,141.7370,41,hkw,2016-07-25 02:00:00,11.3776,0.0000
2,20160725,200,0,2,-37.7155,3.5175,8.9233,78.2254,39.3297,-147.2205,11.3467,44.4252,141.2232,41,hkw,2016-07-25 02:00:00,11.3467,3.5175
3,20160725,200,0,3,-39.7328,6.5212,8.5291,46.0231,39.6014,-147.6740,11.3165,44.6902,140.7215,41,hkw,2016-07-25 02:00:00,11.3165,6.5212
4,20160725,200,0,4,-41.4199,3.6386,8.9351,34.0043,39.8590,-148.1218,11.2866,44.9400,140.2269,41,hkw,2016-07-25 02:00:00,11.2866,3.6386


In [9]:
# read from the feather file
lshellDF = feather.read_dataframe('../data/lshellFits-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
lshellDF.head()

,azimSAPS,azimSTD,delMLT,goodFit,mlat,normMlt,velSAPS,velSTD,endPtMLAT,endPtNormMLT,date
0,-3.236139,1.387087,0.5,True,62.0,-5.0,646.002636,43.323736,62.04,-5.64,2016-07-25 04:06:00
1,-5.878198,2.760037,1.5,True,61.0,-4.0,620.821279,57.591342,61.06,-4.62,2016-07-25 04:06:00
2,-8.114615,3.692693,1.5,True,61.5,-5.0,514.861281,62.939880,61.57,-5.51,2016-07-25 04:06:00
3,-6.618126,1.352016,1.0,True,61.0,-5.0,422.839892,19.427486,61.05,-5.42,2016-07-25 04:06:00
4,-11.870053,2.809972,1.0,True,61.0,-3.0,749.034029,65.355440,61.15,-3.73,2016-07-25 04:06:00


In [10]:
# read TEC data from the feather file
tecDF = feather.read_dataframe('../data/tec-mf-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
tecDF.head()

,dateStr,timeStr,Mlat,Mlon,med_tec,dlat,dlon,date,MLT
0,20160725,200,11.0,0.0,3.1,1.0,2.0,2016-07-25 02:00:00,21.165544
1,20160725,200,11.0,2.0,2.7,1.0,2.0,2016-07-25 02:00:00,21.298877
2,20160725,200,11.0,4.0,2.3,1.0,2.0,2016-07-25 02:00:00,21.432210
3,20160725,200,11.0,6.0,2.7,1.0,2.0,2016-07-25 02:00:00,21.565544
4,20160725,200,11.0,8.0,2.7,1.0,2.0,2016-07-25 02:00:00,21.698877
